In [46]:
import numpy as np
from lifelines import CoxPHFitter
from lifelines.datasets import load_rossi

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Fit Rossi with lifelines

In [47]:
rossi = load_rossi()

cph = CoxPHFitter()
cph.fit(rossi, duration_col='week', event_col='arrest')

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 432 total observations, 318 right-censored observations>
             duration col = 'week'
                event col = 'arrest'
      baseline estimation = breslow
   number of observations = 432
number of events observed = 114
   partial log-likelihood = -658.75
         time fit was run = 2022-02-06 10:39:02 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
fin        -0.38       0.68       0.19            -0.75            -0.00                 0.47                 1.00
age        -0.06       0.94       0.02            -0.10            -0.01                 0.90                 0.99
race        0.31       1.37       0.31            -0.29             0.92                 0.75                 2.50
wexp       -0.15       0.86       0.21            -0.57             0.27                 0.57                 1.30
mar        -0.43       0.65       0.38            -1.18             0.31                 0.31                 1.37
paro       -0.08       0.92       0.20            -0.47             0.30                 0.63                 1.35
prio        0.09       1.10       0.03             0.04             0.15                 1.04                 1.16

              z      p   -log2(p)
covariate                        
fin       -1.98   0.05       4.40
age       -2.61   0.01       6.79
race       1.02   0.31       1.70
wexp      -0.71   0.48       1.06
mar       -1.14   0.26       1.97
paro      -0.43   0.66       0.59
prio       3.19 <0.005       9.48
---
Concordance = 0.64
Partial AIC = 1331.50
log-likelihood ratio test = 33.27 on 7 df
-log2(p) of ll-ratio test = 15.37

In [48]:
baseline_hazard = cph.baseline_hazard_['baseline hazard']
coefs = cph.params_

# Fit with CRM and Coxwrapper

In [49]:
from pymsm.competing_risks_model import CompetingRisksModel
from pymsm.event_specific_fitter import CoxWrapper

crm = CompetingRisksModel(event_specific_fitter=CoxWrapper)

crm.fit(rossi, duration_col='week', event_col='arrest')

crm.print_summary()

>>> Fitting Transition to State: 1, n events: 114
Model for failure type 1:



<lifelines.CoxPHFitter: fitted with 432 total observations, 318 right-censored observations>
             duration col = 'week'
                event col = 'arrest'
      baseline estimation = breslow
   number of observations = 432
number of events observed = 114
   partial log-likelihood = -654.11
         time fit was run = 2022-02-06 10:39:02 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
fin        -0.37       0.69       0.19            -0.74             0.01                 0.48                 1.01
age        -0.06       0.94       0.02            -0.10            -0.02                 0.90                 0.99
race        0.32       1.37       0.31            -0.29             0.92                 0.75                 2.51
wexp       -0.14       0.87       0.21            -0.56             0.28                 0.57                 1.32
mar        -0.43       0.65       0.38            -1.18             0.32                 0.31                 1.37
paro       -0.07       0.93       0.20            -0.46             0.31                 0.63                 1.36
prio        0.09       1.10       0.03             0.04             0.15                 1.04                 1.16

              z      p   -log2(p)
covariate                        
fin       -1.92   0.05       4.20
age       -2.64   0.01       6.93
race       1.03   0.30       1.73
wexp      -0.66   0.51       0.97
mar       -1.13   0.26       1.96
paro      -0.38   0.70       0.51
prio       3.27 <0.005       9.84
---
Concordance = 0.64
Partial AIC = 1322.21
log-likelihood ratio test = 33.56 on 7 df
-log2(p) of ll-ratio test = 15.55

# Fit with CRM and ManualCoxwrapper

In [64]:
from pymsm.competing_risks_model import CompetingRisksModel, EventSpecificModel
from pymsm.event_specific_fitter import ManualCoxWrapper

crm = CompetingRisksModel(event_specific_fitter=ManualCoxWrapper)
crm.event_specific_models = {
    1: EventSpecificModel(
        failure_type=1, model=ManualCoxWrapper(coefs, baseline_hazard)
    )
}


crm.event_specific_models[1].model.print_summary()


Manual cox model
Coefficients: [-0.37942216 -0.05743772  0.31389978 -0.14979572 -0.43370385 -0.08487107
  0.09149708]


# MSM with CoxWrapper

In [86]:
covariate_names = ['fin', 'age', 'race', 'wexp', 'mar', 'paro', 'prio']
rossi_competing_risk_data = rossi[covariate_names].copy()
rossi_competing_risk_data['sample_id'] = rossi_competing_risk_data.index.values
rossi_competing_risk_data['origin_state'] = 1
rossi_competing_risk_data['target_state'] = rossi['arrest'].replace({1:2})
rossi_competing_risk_data['time_entry_to_origin'] = 0
rossi_competing_risk_data['time_transition_to_target'] = rossi['week']

rossi_competing_risk_data.head()

,fin,age,race,wexp,mar,paro,prio,sample_id,origin_state,target_state,time_entry_to_origin,time_transition_to_target
0,0,27,1,0,0,1,3,0,1,2,0,20.000000
1,0,18,1,0,0,1,8,1,1,2,0,17.000095
2,0,19,0,1,0,1,13,2,1,2,0,25.000073
3,1,23,1,1,1,1,1,3,1,0,0,52.000060
4,0,19,0,1,0,1,3,4,1,0,0,52.000016


In [87]:
from pymsm.multi_state_competing_risks_model import (
    MultiStateModel,
    default_update_covariates_function,
)

terminal_states = [1]
update_covariates_fn=default_update_covariates_function

msm = MultiStateModel(dataset=rossi_competing_risk_data,
            terminal_states=terminal_states,
            update_covariates_fn=update_covariates_fn,
            covariate_names=covariate_names,
            event_specific_fitter=CoxWrapper,
            competing_risk_data_format=True)


msm.fit()


Fitting Model at State: 1
>>> Fitting Transition to State: 2, n events: 114


# MSM with ManualCoxWrapper

In [88]:
covariate_names = ['fin', 'age', 'race', 'wexp', 'mar', 'paro', 'prio']
rossi_competing_risk_data = rossi[covariate_names].copy()
rossi_competing_risk_data['sample_id'] = rossi_competing_risk_data.index.values
rossi_competing_risk_data['origin_state'] = 1
rossi_competing_risk_data['target_state'] = rossi['arrest'].replace({1:2})
rossi_competing_risk_data['time_entry_to_origin'] = 0
rossi_competing_risk_data['time_transition_to_target'] = rossi['week']

rossi_competing_risk_data.head()

,fin,age,race,wexp,mar,paro,prio,sample_id,origin_state,target_state,time_entry_to_origin,time_transition_to_target
0,0,27,1,0,0,1,3,0,1,2,0,20.000000
1,0,18,1,0,0,1,8,1,1,2,0,17.000095
2,0,19,0,1,0,1,13,2,1,2,0,25.000073
3,1,23,1,1,1,1,1,3,1,0,0,52.000060
4,0,19,0,1,0,1,3,4,1,0,0,52.000016


In [96]:
from pymsm.multi_state_competing_risks_model import (
    MultiStateModel,
    default_update_covariates_function,
)

terminal_states = [1]
update_covariates_fn = default_update_covariates_function

msm = MultiStateModel(
    dataset=rossi_competing_risk_data,
    terminal_states=terminal_states,
    update_covariates_fn=update_covariates_fn,
    covariate_names=covariate_names,
    event_specific_fitter=ManualCoxWrapper,
    competing_risk_data_format=True,
)


crm_1 = CompetingRisksModel(event_specific_fitter=ManualCoxWrapper)
crm_1.event_specific_models = {
    1: EventSpecificModel(
        failure_type=1, model=ManualCoxWrapper(coefs, baseline_hazard)
    )
}


msm.state_specific_models = {1: crm_1}
msm.state_specific_models[1].extract_necessary_attributes()


msm.run_monte_carlo_simulation(
    sample_covariates=rossi_competing_risk_data.loc[0, covariate_names].values,
    origin_state=1,
    current_time=0,
    n_random_samples=3,
    max_transitions=10,
)



KeyError: None

# Define MSMSim

In [5]:
#  terminal_states=[],
# update_covariates_fn=default_update_covariates_function,
# covariate_names=None,
# state_specific_models=dict(),
# covariate_data=None,

In [8]:
coefs.head()

covariate
fin    -0.379422
age    -0.057438
race    0.313900
wexp   -0.149796
mar    -0.433704
Name: coef, dtype: float64

In [14]:
from pymsm.event_specific_fitter import ManualCoxWrapper

state_specific_models = {1: (coefs,baseline_hazard)}

state_specific_models = {k:ManualCoxWrapper(*v) for k,v in state_specific_models.items()}

state_specific_models

{1: <pymsm.event_specific_fitter.ManualCoxWrapper at 0x7fd52e2d39a0>}

In [15]:

model = state_specific_models[1]

In [16]:
model.get_coefficients()

array([-0.37942216, -0.05743772,  0.31389978, -0.14979572, -0.43370385,
       -0.08487107,  0.09149708])

# XXX

In [44]:
from lifelines import CoxPHFitter
from lifelines.datasets import load_rossi

rossi = load_rossi()
cph = CoxPHFitter()
cph.fit(rossi, duration_col='week', event_col='arrest')

print(cph.baseline_hazard_.head())
print(cph.baseline_cumulative_hazard_.head())
print(cph.baseline_cumulative_hazard_.head().diff().fillna(cph.baseline_cumulative_hazard_.iloc[0]))

     baseline hazard
1.0         0.001958
2.0         0.001964
3.0         0.001965
4.0         0.001969
5.0         0.001975
     baseline cumulative hazard
1.0                    0.001958
2.0                    0.003922
3.0                    0.005887
4.0                    0.007856
5.0                    0.009832
     baseline cumulative hazard
1.0                    0.001958
2.0                    0.001964
3.0                    0.001965
4.0                    0.001969
5.0                    0.001975


In [ ]:
# Write an abstract
